In [25]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, accuracy_score,precision_score,recall_score
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.model_selection import GridSearchCV,cross_val_score,KFold
from sklearn.neural_network import MLPClassifier
import matplotlib.pylab as py
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

In [2]:
#new_store = pd.read_hdf("singlepi_e100GeV_pu200Nov7.h5")
store_oct27 = pd.read_hdf("singlepi_e100GeV_pu200_oct27.h5")

In [3]:
# Convert the data to 2 target classes (pile-up, non-pileup.) instead of 3 levels of purity.
store_oct27['purity']=store_oct27['purity'].apply(lambda x: 0 if x <=1 else 1 )

In [4]:
# drop highly correlated values
df = store_oct27.drop(['eta','phi','layer','trckPhi','trckEn','trckEta','trckType'],1,inplace=False)

In [41]:
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(32, input_dim=x0Train.shape[1], activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy'])
    return model

In [42]:
# Train and Test
trainDF=df.sample(frac=0.85,random_state=1234) #random state is a seed value
testDF=df.drop(trainDF.index)

x0Train = trainDF.drop(['purity','event','trackster'],1,inplace=False)
x0Test = testDF.drop(['purity','event','trackster'],1,inplace=False)

sc = StandardScaler()
X_train = sc.fit_transform(x0Train)
X_test = sc.transform(x0Test)

y_train = trainDF[['purity']].iloc[:,0]
y_test =  testDF[['purity']].iloc[:,0]

In [43]:
model = baseline_model()
model.fit(X_train, y_train,nb_epoch=300,batch_size=32)

C:\Users\20184731\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Epoch 1/300
27433/27433 [==============================] - 1s 34us/step - loss: 0.2462 - sparse_categorical_accuracy: 0.9001
Epoch 2/300
27433/27433 [==============================] - 1s 32us/step - loss: 0.2040 - sparse_categorical_accuracy: 0.9155
Epoch 3/300
27433/27433 [==============================] - 1s 33us/step - loss: 0.1984 - sparse_categorical_accuracy: 0.9176
Epoch 4/300
27433/27433 [==============================] - 1s 27us/step - loss: 0.1952 - sparse_categorical_accuracy: 0.9194
Epoch 5/300
27433/27433 [==============================] - 1s 31us/step - loss: 0.1935 - sparse_categorical_accuracy: 0.9200
Epoch 6/300
27433/27433 [==============================] - 1s 33us/step - loss: 0.1922 - sparse_categorical_accuracy: 0.9197
Epoch 7/300
27433/27433 [==============================] - 1s 34us/step - loss: 0.1913 - sparse_categorical_accuracy: 0.9209
Epoch 8/300
27433/27433 [==============================] - 1s 32us/step - loss: 0.1909 - sparse_categorical_accuracy: 0.9207


In [44]:
pred2=model.predict_classes(X_test, batch_size=32, verbose=0)
print("Test accuracy")
print(accuracy_score(y_test, pred2))
print("Test F1 score")
print(f1_score(y_test, pred2))
print("Testing confusion_matrix")
print(confusion_matrix(y_test, pred2))

Test accuracy
0.9287337326998554
Test F1 score
0.8094975151849806
Testing confusion_matrix
[[3763  163]
 [ 182  733]]


In [51]:
threshold = 0.98
pred2_proba=model.predict_proba(X_test, batch_size=32, verbose=0)

pred2_probe_threshold = (pred2_proba [:,1] >= threshold).astype('int')

print("Test accuracy",end =": ")
print(accuracy_score(y_test, pred2_probe_threshold))
print("Test F1 score",end =": ")
print(f1_score(y_test, pred2_probe_threshold))
print("Test recall_score",end =": ")
print(recall_score(y_test, pred2_probe_threshold))
print("##Test precision_score##",end =": ")
print(precision_score(y_test, pred2_probe_threshold))
print("Testing confusion_matrix")
print(confusion_matrix(y_test, pred2_probe_threshold))

Test accuracy: 0.8929973146044206
Test F1 score: 0.6140089418777943
Test recall_score: 0.45027322404371584
##Test precision_score##: 0.9648711943793911
Testing confusion_matrix
[[3911   15]
 [ 503  412]]


In [52]:
threshold = 0.02
pred2_proba=model.predict_proba(X_test, batch_size=32, verbose=0)

pred2_probe_threshold = (pred2_proba [:,1] >= threshold).astype('int')

print("Test accuracy",end =": ")
print(accuracy_score(y_test, pred2_probe_threshold))
print("Test F1 score",end =": ")
print(f1_score(y_test, pred2_probe_threshold))
print("Test recall_score",end =": ")
print(recall_score(y_test, pred2_probe_threshold))
print("Test precision_score",end =": ")
print(precision_score(y_test, pred2_probe_threshold))
print("Testing confusion_matrix")
print(confusion_matrix(y_test, pred2_probe_threshold))

Test accuracy: 0.796323073745094
Test F1 score: 0.6414545454545454
##Test recall_score##: 0.9639344262295082
Test precision_score: 0.48065395095367847
Testing confusion_matrix
[[2973  953]
 [  33  882]]


In [49]:
pred2_proba[1:20]

array([[9.9998069e-01, 1.9351208e-05],
       [9.9999714e-01, 2.8973509e-06],
       [9.6703064e-01, 3.2969389e-02],
       [9.9999738e-01, 2.6513831e-06],
       [9.9987543e-01, 1.2460476e-04],
       [1.0000000e+00, 1.3021791e-13],
       [1.0000000e+00, 1.7375278e-08],
       [9.2915815e-01, 7.0841804e-02],
       [9.9999964e-01, 3.0739571e-07],
       [9.9999285e-01, 7.1187260e-06],
       [9.9471647e-01, 5.2835387e-03],
       [7.1431965e-01, 2.8568035e-01],
       [2.4791721e-15, 1.0000000e+00],
       [9.9964511e-01, 3.5493140e-04],
       [9.9517041e-01, 4.8295385e-03],
       [7.2006828e-01, 2.7993178e-01],
       [8.4006172e-01, 1.5993826e-01],
       [9.9956983e-01, 4.3020901e-04],
       [9.6942544e-01, 3.0574609e-02]], dtype=float32)

In [50]:
pred2_probe_threshold[1:20]

array([0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1])